In [1]:
import tensorflow as tf
import keras_tuner
import numpy as np


import pandas as pd
import openpyxl # Needed for reading excel
import pathlib

import decomposition
import models
import data
from metrics import smape
from windower import WindowGenerator
import hp_training
import results

2023-03-10 13:09:37.921219: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [157]:
cwd = pathlib.Path.cwd()

code_directory = cwd.parents[1]
gonem_directory = code_directory / "notebooks" / "Gonem"
model_directory = gonem_directory / 'models'

model = 'ARF'
product = 'wheat'
model_path =  model_directory / f"{model}_{product}_in_sample"
checkpoint_path = model_path / f"{model}_{product}"


In [158]:
df = data.get_data(directory_path=gonem_directory, product=product)
df = df.iloc[:-2]
df.describe()

AVG_TAVG                                                     \
PARTNER_Labels     Belgium      France     Germany     Romania United Kingdom   
count           214.000000  214.000000  214.000000  214.000000     214.000000   
mean            111.302520  125.494113  104.143755  105.397067      96.544608   
std              57.100073   55.456797   65.178828   81.500731      39.315210   
min              -0.733333   20.168095  -29.860742  -65.129397       7.340256   
25%              63.525974   77.127932   48.280935   29.836943      61.896038   
50%             104.345960  121.548459  102.466712  104.869216      96.188597   
75%             163.724856  177.640994  163.845330  182.880379     131.974731   
max             239.666667  231.747995  229.864177  243.079179     178.133333   

                  MAX_TMAX                                                     \
PARTNER_Labels     Belgium      France     Germany     Romania United Kingdom   
count           214.000000  214.000000  214.000000  214.000000     214.000000   
mean            215.733645  225.531663  218.804582  224.761987     168.507655   
std              79.022190   67.153277   81.278888   81.711107      50.004673   
min              59.000000  107.622222   36.388889   47.000000      74.500000   
25%             147.500000  163.656487  140.181174  157.733333     124.583333   
50%             223.000000  228.556851  226.650735  229.750000     164.437500   
75%             281.000000  284.822581  287.334967  300.500000     208.409091   
max             397.000000  362.382979  371.823529  367.750000     300.562500   

                ... renewable_energy_consumption_perc_of_total              \
PARTNER_Labels  ...                                    Belgium      France   
count           ...                                 214.000000  214.000000   
mean            ...                                   7.672173   12.883762   
std             ...                                   2.698506    2.273139   
min             ...                                   2.460000    8.520000   
25%             ...                                   5.438125   11.155000   
50%             ...                                   9.016250   13.286667   
75%             ...                                  10.210000   15.310000   
max             ...                                  10.660000   15.530000   

                                                      unemployment_total  \
PARTNER_Labels     Germany     Romania United Kingdom            Belgium   
count           214.000000  214.000000     214.000000         214.000000   
mean             13.639136   22.297593       7.120444           7.272757   
std               2.851412    2.019432       3.915944           0.994389   
min               7.280000   17.390000       1.350000           5.360000   
25%              11.109375   21.416250       3.506875           6.260000   
50%              13.971250   23.290833       7.192500           7.460000   
75%              16.448125   23.540000      11.380625           8.149583   
max              17.170000   24.400000      12.240000           8.520000   

                                                                   
PARTNER_Labels      France     Germany     Romania United Kingdom  
count           214.000000  214.000000  214.000000     214.000000  
mean              8.969860    5.587944    6.091075       5.505070  
std               0.867082    2.216610    0.996584       1.547705  
min               7.390000    3.140000    3.910000       3.740000  
25%               8.086250    3.646250    5.508333       3.918750  
50%               9.068750    5.011250    6.412083       5.278750  
75%               9.789167    7.403125    6.926250       7.140625  
max              10.350000   11.170000    7.270000       8.040000  

[8 rows x 63 columns]

In [159]:
label_columns = ['price']
label_columns = df.columns[df.columns.get_level_values(0).isin(label_columns)].tolist()
label_columns

[('price', 'Belgium'),
 ('price', 'France'),
 ('price', 'Germany'),
 ('price', 'Global'),
 ('price', 'Romania'),
 ('price', 'United Kingdom')]

In [160]:
stl = decomposition.STLDecomposer(labels=label_columns, period=12)
log = decomposition.Logger(labels=label_columns)
std = decomposition.Standardizer()

preproc = decomposition.Processor().add(stl).add(log).add(std)

In [161]:
width = 24
label_width = 6
shift = 6

window = WindowGenerator(input_width=width, label_width=label_width, shift=shift, data=df, 
                    # train_begin=0, train_end=.9, val_begin=None, val_end=.96,
                    train_begin=0, train_end=.97, val_begin=None, val_end=None,
                    # train_begin=0, train_end=.5, val_begin=None, val_end=.8,
                    test_begin=0., test_end=1., connect=True, remove_labels=True, label_columns=label_columns)
window.preprocess(preproc)
window

Total window size: 30
Input indices: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23]
Label indices: [24 25 26 27 28 29]
Label column name(s): [('price', 'Belgium'), ('price', 'France'), ('price', 'Germany'), ('price', 'Global'), ('price', 'Romania'), ('price', 'United Kingdom')]

In [162]:
label_std = decomposition.Standardizer(mean=std.mean[window.label_columns], std=std.std[window.label_columns])
label_log = decomposition.Logger(label_indices=range(len(window.label_columns)))
postproc = decomposition.Processor().add(label_std).add(label_log)
window.add_label_postprocess(postproc)

In [163]:
tuner = hp_training.get_tuner(model, gonem_directory/'hp', window)

INFO:tensorflow:Reloading Tuner from /code/notebooks/Gonem/hp/ARF/tuner0.json


In [164]:
# hp_training.run(tuner, window)

In [165]:
best_hps = tuner.get_best_hyperparameters()[0]
best_hps.values

{'lstm_units': 224,
 'lstm_layers': 1,
 'prediction_units': 64,
 'prediction_layers': 6,
 'feature_units': 416,
 'feature_layers': 8,
 'heads': 5,
 'dropout': 0.18732635985077017,
 'key_dim': 112,
 'l1': 0.0009003172160791668,
 'l2': 0.00010979313621486196,
 'learning_rate': 0.000291383964865669,
 'tuner/epochs': 200,
 'tuner/initial_epoch': 67,
 'tuner/bracket': 2,
 'tuner/round': 2,
 'tuner/trial_id': '0200'}

In [166]:
hp_training.final_train(tuner, window, checkpoint_path)

Epoch 1/200
6/6 - 12s - loss: 215.2585 - mae: 0.7864 - mse: 0.9938 - mape: 140.5968 - smape: 160.5841 - val_loss: 166.5733 - val_mae: 0.7571 - val_mse: 0.9659 - val_mape: 263.0664 - val_smape: 113.4517 - 12s/epoch - 2s/step
Epoch 2/200
6/6 - 2s - loss: 176.2173 - mae: 0.7821 - mse: 1.0068 - mape: 247.9312 - smape: 122.5480 - val_loss: 125.0605 - val_mae: 0.5064 - val_mse: 0.6103 - val_mape: 205.3901 - val_smape: 72.0818 - 2s/epoch - 392ms/step
Epoch 3/200
6/6 - 2s - loss: 159.0959 - mae: 0.6678 - mse: 0.8443 - mape: 221.2731 - smape: 106.0408 - val_loss: 127.8707 - val_mae: 0.5695 - val_mse: 0.7326 - val_mape: 197.5930 - val_smape: 74.2263 - 2s/epoch - 342ms/step
Epoch 4/200
6/6 - 2s - loss: 160.4147 - mae: 0.6224 - mse: 0.7672 - mape: 196.1225 - smape: 107.4844 - val_loss: 129.8403 - val_mae: 0.6384 - val_mse: 0.8046 - val_mape: 295.0892 - val_smape: 77.0163 - 2s/epoch - 339ms/step
Epoch 5/200
6/6 - 2s - loss: 157.0121 - mae: 0.7388 - mse: 0.9774 - mape: 293.7535 - smape: 104.7738 - v

In [167]:
m = tuner.hypermodel.build(best_hps)
m.load_weights(checkpoint_path)
m.evaluate(window.test)

6/6 [==============================] - 2s 89ms/step - loss: 128.9904 - mae: 0.5178 - mse: 0.6161 - mape: 222.8627 - smape: 76.8236


[128.99041748046875,
 0.5177812576293945,
 0.6160849928855896,
 222.86273193359375,
 76.82357025146484]

In [168]:
# w.test

# # val_performance['1'] = m.evaluate(w.val)
# for i in range(6):

#     label = label_columns[i]
#     print(label)
#     # performance['1'] = m.evaluate(w.test)
#     w.plot(m, plot_col=label, max_subplots=7)


In [169]:
inputs, labels, predictions, weights, mcds = [], [], [], [], []
for x, y in window.test.take(40):
    inputs.append(x)
    lab = y
    lab = window.label_postprocessor.reverse(lab)
    labels.append(lab)
    
    pred = m(x)
    pred = window.label_postprocessor.reverse(pred)
    predictions.append(pred)
    
    weight = m.attention_layer(x, return_weights=True)[1]
    weights.append(weight)
    
    mcd = results.monte_carlo_dropout(x, m, 100, window.label_postprocessor.reverse, return_weight=False)
    mcds.append(mcd)
    weights.append(weight)
    
inputs = tf.concat(inputs, axis=0)
labels = tf.concat(labels, axis=0)
weights = tf.concat(weights, axis=0)
weights = tf.reduce_mean(weights, axis=0)
predictions = tf.concat(predictions, axis=0)
mcds = tf.concat(mcds, axis=1)

In [170]:
np.save(model_path / f"{product}_inputs", inputs.numpy())
np.save(model_path / f"{product}_labels", labels.numpy())
np.save(model_path / f"{product}_weights", weights.numpy())
np.save(model_path / f"{product}_predictions", predictions.numpy())
np.save(model_path / f"{product}_mcd_predictions", mcds.numpy())


In [100]:
predictions.shape

TensorShape([185, 6, 6])